In [12]:
import pandas as pd
import numpy as np
import os
from tqdm import tnrange,tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
os.chdir("C:\\workspace\\DACON_3\\Auction_master_kr")
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

AM_train = pd.read_csv("Auction_master_train.csv")
AM_test = pd.read_csv("Auction_master_test.csv")
AM_test.drop(index=[397], axis = 0, inplace=True)
#AM_train = pd.concat([AM_train, AM_test], axis = 0)
AM_train.shape

# del AM_train["point.x"]
# del AM_train["point.y"]
del AM_train['Appraisal_company']
del AM_train["Final_result"]
del AM_train["addr_si"]
del AM_train["addr_dong"]
del AM_train["addr_li"]
del AM_train["addr_san"]
del AM_train["addr_bunji1"]
del AM_train["addr_bunji2"]
del AM_train["addr_etc"]
del AM_train["Specific"]
del AM_train["road_name"]
del AM_train["road_bunji1"]
del AM_train["road_bunji2"]
del AM_train["Close_result"]
del AM_train["Creditor"]
###############################################################################

#del AM_train['Total_land_gross_area']






import datetime

def change_datetime(x):
    today = datetime.date.today()
    someday = datetime.date(int(x[:4]), int(x[5:7]), int(x[8:10]))
    diff = today - someday
    return diff.days

# del AM_train["Appraisal_date"]
# del AM_train["First_auction_date"]
# del AM_train["Preserve_regist_date"]
# del AM_train["Final_auction_date"]
# del AM_train["Close_date"]
AM_train["Appraisal_date"] = AM_train["Appraisal_date"].apply(lambda x : x[:10])
AM_train["First_auction_date"] = AM_train["First_auction_date"].apply(lambda x : x[:10])
AM_train["Final_auction_date"] = AM_train["Final_auction_date"].apply(lambda x : x[:10])
AM_train["Preserve_regist_date"] = AM_train["Preserve_regist_date"].apply(lambda x : x[:10])
AM_train["Close_date"] = AM_train["Close_date"].apply(lambda x : x[:10])
AM_train["Appraisal_date"] = AM_train["Appraisal_date"].apply(change_datetime)
AM_train["First_auction_date"] = AM_train["First_auction_date"].apply(change_datetime)
AM_train["Final_auction_date"] = AM_train["Final_auction_date"].apply(change_datetime)
AM_train["Preserve_regist_date"] = AM_train["Preserve_regist_date"].apply(change_datetime)
AM_train["Close_date"] = AM_train["Close_date"].apply(change_datetime)

AM_train["Diff_price"] = AM_train['Total_appraisal_price'] - AM_train['Minimum_sales_price']
AM_train["Diff_date"] = AM_train['First_auction_date'] - AM_train['Final_auction_date']

print(AM_train.Auction_class.unique())
AM_train.Auction_class.replace({'임의':0, '강제':1},inplace=True)
print(AM_train.Auction_class.unique())
print("-"*50)
print(AM_train.addr_do.unique())
AM_train.addr_do.replace({'부산':0, '서울':1},inplace=True)
print(AM_train.addr_do.unique())
print("-"*50)
print(AM_train.Apartment_usage.unique())
AM_train.Apartment_usage.replace({'주상복합':0, '아파트':1},inplace=True)
print(AM_train.Apartment_usage.unique())
print("-"*50)
print(AM_train.Share_auction_YorN.unique())
AM_train.Share_auction_YorN.replace({'N':0, 'Y':1},inplace=True)
print(AM_train.Share_auction_YorN.unique())
print("-"*50)
def change_spec(x):
    if isinstance(x, str):
        return int(1)
    else:
        return int(0)
# AM_train.Creditor = AM_train.Creditor.apply(lambda x : x if x == 'Private' else 'Company')
# print(AM_train.Creditor.unique())
# AM_train.Creditor.replace({'Private':0, 'Company':1},inplace=True)
# print(AM_train.Creditor.unique())
#AM_train["Specific"] = AM_train.Specific.apply(change_spec)
#print(AM_train.Specific.unique())

del AM_train["Preserve_regist_date"]
#del AM_train["Final_auction_date"]
del AM_train["First_auction_date"]
del AM_train["Close_date"]
#del AM_train["Appraisal_date"]
AM_train.Hammer_price = AM_train.Hammer_price.apply(lambda x : 1 if x >= AM_train.Hammer_price.median() else 0)

    
cate_features = AM_train.dtypes[AM_train.dtypes == "object"].index.format()
conti_featrues = AM_train.dtypes[AM_train.dtypes != "object"].index.format()

conti_featrues.remove("Hammer_price")
conti_featrues.remove("Auction_key")

print("Continuous Feature ",conti_featrues)
print("Categorical Feature ",cate_features)
print("-"*50)


from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
AM_train[conti_featrues] = scale.fit_transform(AM_train[conti_featrues])
X = pd.concat([AM_train[conti_featrues], pd.get_dummies(AM_train[cate_features])], axis= 1)
y = AM_train.Hammer_price
X = pd.concat([X,y], axis=1)

print("Shape of X is ", X.shape)
print("-"*50)

import pickle
os.chdir("C:\\workspace\\DACON_3\\wk2")

with open('AM_train.pickle','wb') as f:
    pickle.dump(X,f,pickle.HIGHEST_PROTOCOL)
    

print("Finish!!!")

['임의' '강제']
[0 1]
--------------------------------------------------
['부산' '서울']
[0 1]
--------------------------------------------------
['주상복합' '아파트']
[0 1]
--------------------------------------------------
['N' 'Y']
[0 1]
--------------------------------------------------
Continuous Feature  ['Auction_class', 'Claim_price', 'Appraisal_date', 'Auction_count', 'Auction_miscarriage_count', 'Total_land_gross_area', 'Total_land_real_area', 'Total_land_auction_area', 'Total_building_area', 'Total_building_auction_area', 'Total_appraisal_price', 'Minimum_sales_price', 'Final_auction_date', 'addr_do', 'Apartment_usage', 'Total_floor', 'Current_floor', 'Share_auction_YorN', 'point.y', 'point.x', 'Diff_price', 'Diff_date']
Categorical Feature  ['Bid_class']
--------------------------------------------------
Shape of X is  (1933, 26)
--------------------------------------------------
Finish!!!
